# Playing with NBA API

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Player career stats
Simple example showing how to look up particular stats for a player. 

First, have to look up the player from the static API. This has the player's ID, name, and whether they are active. You can use this to look up a player, or get every player (active, inactive, or either)

In [5]:
from nba_api.stats.static import players
player_info = players.find_players_by_full_name('Kevin Durant') #can also search by player number, first name, last name, etc.
player_info

[{'id': 201142,
  'full_name': 'Kevin Durant',
  'first_name': 'Kevin',
  'last_name': 'Durant',
  'is_active': True}]

It's easier now to move to looking up players by ID number, so jot down the number

In [6]:
id = player_info[0]['id']

Moving forward, we want to look up stats (that may be changing frequently) from the dynamic API endpoints. There are a lot of endpoints and the table of contents can be found [here](https://github.com/swar/nba_api/blob/master/docs/table_of_contents.md).

In [9]:
from nba_api.stats.endpoints import playercareerstats
career = playercareerstats.PlayerCareerStats(id) # first, get KD's career stats from is ID

per = 'PerGame' #per mode can be 'Totals', 'PerGame', or 'Per36'
stats = playercareerstats.PlayerCareerStats(id, per_mode36=per).get_data_frames()[0] #get the stats, and convert to a dataframe
stats.head()

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,201142,2007-08,00,1610612760,SEA,19.0,80,80,34.6,7.3,...,0.873,0.9,3.5,4.4,2.4,1.0,0.9,2.9,1.5,20.3
1,201142,2008-09,00,1610612760,OKC,20.0,74,74,39.0,8.9,...,0.863,1.0,5.5,6.5,2.8,1.3,0.7,3.0,1.8,25.3
2,201142,2009-10,00,1610612760,OKC,21.0,82,82,39.5,9.7,...,0.900,1.3,6.3,7.6,2.8,1.4,1.0,3.3,2.1,30.1
3,201142,2010-11,00,1610612760,OKC,22.0,78,78,38.9,9.1,...,0.880,0.7,6.1,6.8,2.7,1.1,1.0,2.8,2.0,27.7
4,201142,2011-12,00,1610612760,OKC,23.0,66,66,38.6,9.7,...,0.860,0.6,7.4,8.0,3.5,1.3,1.2,3.8,2.0,28.0


In [ ]:
stats['SEASON_ID']

In [ ]:
#plot
stat = 'PTS'
plt.plot(stats['SEASON_ID'], stats[stat])
plt.xlabel('Year')
plt.xticks(rotation = 45, horizontalalignment='right')
plt.ylabel(stat + ' (' + per + ')')
plt.title(player_info[0]['full_name'] + '\'s ' + stat)
plt.grid(True)
plt.show()      

## Team stats

You can also import the teams from the static API and look up team ID and info. Lets plot some stuff for the Knicks

In [ ]:
from nba_api.stats.static import teams
knicks = teams.find_teams_by_full_name('knicks')
knicks_id = knicks[0]['id']

In [ ]:
from nba_api.stats.endpoints import teamplayerdashboard
knicks_stats = teamplayerdashboard.TeamPlayerDashboard(team_id=knicks_id, per_mode_detailed='PerGame')
knicks_stats = knicks_stats.players_season_totals.get_data_frame()

from adjustText import adjust_text
fig, ax = plt.subplots(figsize=(7,7))
x = knicks_stats['FT_PCT']
y = knicks_stats['FG_PCT']
namelist = knicks_stats['NICKNAME']

plt.scatter(x,y)
texts = []
for x, y, s in zip(x, y, namelist):
    texts.append(plt.text(x, y, s))
adjust_text(texts)


ax.set_xlabel('FT PCT')
ax.set_ylabel('FG PCT')

# Getting  player game logs

In [3]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
player_info = players.find_players_by_full_name('Karl-Anthony Towns') #can also search by player number, first name, last name, etc.
player_id = player_info[0]['id']
info = commonplayerinfo.CommonPlayerInfo(player_id).get_data_frames()[0]

In [ ]:
info['POSITION']

## Get base game stats

In [4]:
from nba_api.stats.endpoints import playergamelogs

base_logs = playergamelogs.PlayerGameLogs(player_id_nullable = player_id, season_nullable='2021-22').get_data_frames()[0]
base_logs.columns

Index(['SEASON_YEAR', 'PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID',
       'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MATCHUP',
       'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM',
       'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',
       'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS', 'NBA_FANTASY_PTS', 'DD2',
       'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK',
       'MIN_RANK', 'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK',
       'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK',
       'OREB_RANK', 'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK',
       'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK',
       'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
       'WNBA_FANTASY_PTS_RANK', 'VIDEO_AVAILABLE_FLAG'],
      dtype='object')

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,...,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,VIDEO_AVAILABLE_FLAG
0,2021-22,1626157,Karl-Anthony Towns,Karl-Anthony,1610612750,MIN,Minnesota Timberwolves,0022101199,2022-04-07T00:00:00,MIN vs. SAS,...,26,56,19,49,31,52,1,2,44,1
1,2021-22,1626157,Karl-Anthony Towns,Karl-Anthony,1610612750,MIN,Minnesota Timberwolves,0022101184,2022-04-05T00:00:00,MIN vs. WAS,...,26,6,19,24,64,20,1,2,29,1
2,2021-22,1626157,Karl-Anthony Towns,Karl-Anthony,1610612750,MIN,Minnesota Timberwolves,0022101173,2022-04-03T00:00:00,MIN @ HOU,...,1,56,19,18,39,26,1,2,25,1
3,2021-22,1626157,Karl-Anthony Towns,Karl-Anthony,1610612750,MIN,Minnesota Timberwolves,0022101159,2022-04-01T00:00:00,MIN @ DEN,...,45,56,66,8,38,25,41,2,15,1
4,2021-22,1626157,Karl-Anthony Towns,Karl-Anthony,1610612750,MIN,Minnesota Timberwolves,0022101141,2022-03-30T00:00:00,MIN @ TOR,...,26,56,19,67,62,64,1,2,61,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,2021-22,1626157,Karl-Anthony Towns,Karl-Anthony,1610612750,MIN,Minnesota Timberwolves,0022100087,2021-10-30T00:00:00,MIN vs. DEN,...,45,34,43,71,71,73,41,2,70,1
70,2021-22,1626157,Karl-Anthony Towns,Karl-Anthony,1610612750,MIN,Minnesota Timberwolves,0022100060,2021-10-27T00:00:00,MIN @ MIL,...,1,2,19,28,31,47,41,2,33,1
71,2021-22,1626157,Karl-Anthony Towns,Karl-Anthony,1610612750,MIN,Minnesota Timberwolves,0022100047,2021-10-25T00:00:00,MIN vs. NOP,...,26,15,8,8,28,3,1,2,2,1
72,2021-22,1626157,Karl-Anthony Towns,Karl-Anthony,1610612750,MIN,Minnesota Timberwolves,0022100031,2021-10-23T00:00:00,MIN vs. NOP,...,45,70,66,28,35,58,41,2,49,1


In [5]:
df = base_logs[0][['PLAYER_NAME','GAME_DATE','MATCHUP','PTS','REB','AST']]
df.sort_values('PTS',ascending = False)[:20]

KeyError: 0

## Get advanced game stats

In [ ]:
adv_logs = playergamelogs.PlayerGameLogs(player_id_nullable=player_id, season_nullable='2021-22', measure_type_player_game_logs_nullable='Advanced').get_data_frames()[0]
adv_logs.columns

In [ ]:
adv_logs[['GAME_DATE', 'MATCHUP', 'AST_PCT', 'AST_TO', 'AST_RATIO']][:10]

## Get team opponent stats

In [ ]:
team_id = teams.find_teams_by_full_name('jazz')[0]['id']


In [ ]:
from nba_api.stats.endpoints import playerdashboardbyopponent
a = playerdashboardbyopponent.PlayerDashboardByOpponent(player_id=player_id).get_data_frames()

`dashboard[3]` has the players stats broken up by team.

In [ ]:
a_opp = a[3]
a_opp[a_opp['GROUP_VALUE'] == 'Utah Jazz']

In [ ]:
adv_logs.head()

In [ ]:
base_logs.head()

## Join advanced and base stats

In [10]:
from nba_api.stats.static import teams
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelogs
from nba_api.stats.endpoints import playerdashboardbyopponent
from datetime import datetime
from datetime import timedelta

In [11]:
from nba_api.stats.static import players
player_info = players.find_players_by_full_name('Karl-Anthony Towns') #can also search by player number, first name, last name, etc.
player_info
player_id = player_info[0]['id']

In [12]:
base_logs = playergamelogs.PlayerGameLogs(player_id_nullable = player_id, season_nullable='2021-22').get_data_frames()[0]
adv_logs = playergamelogs.PlayerGameLogs(player_id_nullable=player_id, season_nullable='2021-22', measure_type_player_game_logs_nullable='Advanced').get_data_frames()[0]

#join the advanced and base logs. 'inner' will drop the identical columns e.g. 'PLAYER_NAME', 'TEAM_NAME', etc.
merge_logs = pd.merge(base_logs,adv_logs,on=['PLAYER_ID','GAME_ID'],suffixes=['','_RANK'])
#lets also drop all of the columns with 'RANK' in the name. we don't care about rankings...
merge_logs.drop([i for i in merge_logs.columns if 'RANK' in i], axis=1, inplace=True)


In [13]:
#parse the matchup name to get the opponent abbreviation
merge_logs['OPPONENT'] = merge_logs['MATCHUP'].str.split(' vs. | @ ',expand=True)[1]

In [14]:
#get the opponent ID also
def GetTeamID(s):
    return teams.find_team_by_abbreviation(s)['id']
merge_logs['OPPONENT_ID'] = merge_logs['OPPONENT'].apply(GetTeamID)

In [15]:
merge_logs.head()

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,...,E_PACE,PACE,PACE_PER40,sp_work_PACE,PIE,POSS,FGM_PG,FGA_PG,OPPONENT,OPPONENT_ID
0,2021-22,1626157,Karl-Anthony Towns,Karl-Anthony,1610612750,MIN,Minnesota Timberwolves,0022101199,2022-04-07T00:00:00,MIN vs. SAS,...,110.32,107.94,89.95,107.94,0.100,73,7.0,18.0,SAS,1610612759
1,2021-22,1626157,Karl-Anthony Towns,Karl-Anthony,1610612750,MIN,Minnesota Timberwolves,0022101184,2022-04-05T00:00:00,MIN vs. WAS,...,107.23,106.92,89.10,106.92,0.116,77,10.0,24.0,WAS,1610612764
2,2021-22,1626157,Karl-Anthony Towns,Karl-Anthony,1610612750,MIN,Minnesota Timberwolves,0022101173,2022-04-03T00:00:00,MIN @ HOU,...,107.85,105.71,88.09,105.71,0.161,78,12.0,16.0,HOU,1610612745
3,2021-22,1626157,Karl-Anthony Towns,Karl-Anthony,1610612750,MIN,Minnesota Timberwolves,0022101159,2022-04-01T00:00:00,MIN @ DEN,...,104.79,105.44,87.86,105.44,0.147,69,11.0,18.0,DEN,1610612743
4,2021-22,1626157,Karl-Anthony Towns,Karl-Anthony,1610612750,MIN,Minnesota Timberwolves,0022101141,2022-03-30T00:00:00,MIN @ TOR,...,104.71,102.99,85.82,102.99,0.082,70,7.0,13.0,TOR,1610612761


In [25]:
def GetPriorTeamDefense(row):
    player_id = row['PLAYER_ID']
    opponent_id = row['OPPONENT_ID']
    game_date = row['GAME_DATE']
    
    t = datetime.strptime(game_date,'%Y-%m-%dT%H:%M:%S')
    y = t - timedelta(days=1) #calculate the day before, because you want to look at PRIOR defensive matchups
    a = playerdashboardbyopponent.PlayerDashboardByOpponent(player_id=player_id, opponent_team_id=opponent_id, date_to_nullable=y.strftime('%Y-%m-%dT%H:%M:%S'), per_mode_detailed='PerGame').get_data_frames()[0]
    a.drop([i for i in a.columns if 'RANK' in i], axis=1, inplace=True)
    return a

In [21]:
A = merge_logs.apply(GetPriorTeamDefense,axis=1)
A

In [27]:
B = merge_logs.iloc[:3].apply(GetPriorTeamDefense, axis=1)
B

0    [GROUP_SET, GROUP_VALUE, GP, W, L, W_PCT, MIN,...
1    [GROUP_SET, GROUP_VALUE, GP, W, L, W_PCT, MIN,...
2    [GROUP_SET, GROUP_VALUE, GP, W, L, W_PCT, MIN,...
dtype: object

gradient boosting, random forest, maybe deep learning, linear model with lots of features

In [24]:
list(A[0])

['GROUP_SET',
 'GROUP_VALUE',
 'GP',
 'W',
 'L',
 'W_PCT',
 'MIN',
 'FGM',
 'FGA',
 'FG_PCT',
 'FG3M',
 'FG3A',
 'FG3_PCT',
 'FTM',
 'FTA',
 'FT_PCT',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'TOV',
 'STL',
 'BLK',
 'BLKA',
 'PF',
 'PFD',
 'PTS',
 'PLUS_MINUS',
 'NBA_FANTASY_PTS',
 'DD2',
 'TD3',
 'WNBA_FANTASY_PTS',
 'CFID',
 'CFPARAMS']